In [2]:
print("Ok")

Ok


In [3]:
%pwd

'c:\\PYTHON PROJECT\\research'

In [4]:
import os
os.chdir("../")

In [5]:
%pwd

'c:\\PYTHON PROJECT'

In [5]:
# import os
# os.chdir("C:\\PYTHON PROJECT")


In [6]:
%pwd

'c:\\PYTHON PROJECT'

In [7]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [9]:
extracted_data=load_pdf_file(data='Data/')

In [10]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks
    

In [11]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))


Length of Text Chunks 5802


In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

In [13]:
# pip install sentence-transformers==2.2.2 huggingface_hub==0.11.1


In [14]:
# pip show huggingface_hub


In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

# Load embeddings from local path
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="C:/PYTHON PROJECT/Models/all-MiniLM-L6-v2")
    return embeddings

# Initialize
embeddings = download_hugging_face_embeddings()

# Generate embedding for a query string
query_result = embeddings.embed_query("Hello world")

# Print length of vector (should be 384 for MiniLM)
print("Length:", len(query_result))


C:\Users\VIVEK PANDEY\AppData\Local\Temp\ipykernel_1644\1848003842.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="C:/PYTHON PROJECT/Models/all-MiniLM-L6-v2")
c:\PYTHON PROJECT\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Length: 384


In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [18]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': '383e982385c6c7d0e7a7e32646e40707', 'date': 'Fri, 20 Jun 2025 16:14:28 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [16]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [ ]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [ ]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [ ]:
docsearch

In [ ]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [ ]:
retrieved_docs = retriever.invoke("preamble of constitution?")

In [ ]:
retrieved_docs

In [17]:
## NEW VERSION HERE

import os
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import HuggingFaceEmbeddings

# # Set API key (if not set already)
# os.environ["PINECONE_API_KEY"] = "your-api-key-here"  # Optional if in .env

# Load embeddings model
embeddings = HuggingFaceEmbeddings(model_name="C:/PYTHON PROJECT/Models/all-MiniLM-L6-v2")

# Connect to existing Pinecone index
index_name = "medicalbot"

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

# Use retriever
retriever = docsearch.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

# Retrieve results
query = "preamble of constitution?"
retrieved_docs = retriever.invoke(query)

# # View output
# for i, doc in enumerate(retrieved_docs, 1):
#     print(f"\nResult {i}:\n", doc.page_content)


In [18]:
retrieved_docs

[Document(id='f1273e98-36ba-46e9-b506-070dc6654506', metadata={'creationdate': '', 'creator': 'PyPDF', 'moddate': '2025-06-19T15:56:00+00:00', 'page': 262.0, 'page_label': '263', 'producer': 'iLovePDF', 'source': 'Data\\MERGE.pdf', 'total_pages': 873.0}, page_content='Assembly for the purpose of framing the Constitution  of the State is convened, it shall be placed before such Assembly for such decision  as it may take thereon.(3) Notwithstanding anything in the foregoing provisions of this article, the President may, by public notification, declare that this article shall cease to be operative or shall be operative only with such exceptions and modifications and from such date as  he may specify:Provided that the recommendation of the 1[clause (2) of'),
 Document(id='1195f9d0-9007-43e4-b8ef-965ef5efc34a', metadata={'creationdate': '', 'creator': 'PyPDF', 'moddate': '2025-06-19T15:56:00+00:00', 'page': 262.0, 'page_label': '263', 'producer': 'iLovePDF', 'source': 'Data\\MERGE.pdf', 'to

In [19]:
from dotenv import load_dotenv
load_dotenv()

True

In [20]:
HUGGINGFACEHUB_API_TOKEN=os.environ.get('HUGGINGFACEHUB_API_TOKEN')

In [21]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [ ]:
# pip install huggingface_hub

In [ ]:
# from langchain.llms import HuggingFaceHub
# from langchain.chains import create_retrieval_chain
# from langchain.chains.combine_documents import create_stuff_documents_chain
# from langchain_core.prompts import ChatPromptTemplate

# # 1. Load HuggingFaceHub LLM
# llm = HuggingFaceHub(
#     repo_id="google/flan-t5-base",  # or "tiiuae/falcon-7b-instruct"
#     model_kwargs={"temperature": 0.4, "max_length": 500}
# )


In [ ]:
# import sentence_transformers
# print(sentence_transformers.__version__)


In [44]:
# 1. Load your retriever (from Pinecone or FAISS or any vector store)
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# 2. Load a light Hugging Face model (Flan-T5)
from transformers import pipeline
# pipe = pipeline("text2text-generation", model="google/flan-t5-base")
pipe = pipeline("text2text-generation", model="MBZUAI/LaMini-Flan-T5-783M")


# 3. Helper: Retrieve context
def get_context_from_query(query):
    docs = retriever.invoke(query)
    return "\n".join([doc.page_content for doc in docs])

# 4. Helper: Prompt builder
def build_prompt(context, question):
    return f"""
You are a helpful legal assistant. Use the following context to answer the question clearly and concisely.

Context:
{context}

Question: {question}
"""


c:\PYTHON PROJECT\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\VIVEK PANDEY\.cache\huggingface\hub\models--MBZUAI--LaMini-Flan-T5-783M. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu


In [54]:
# 5. Ask anything now
def ask_question(question):
    context = get_context_from_query(question)
    prompt = build_prompt(context, question)
    response = pipe(prompt, max_new_tokens=200, do_sample=False)
    return response[0]["generated_text"]

In [55]:
print(ask_question("Factors to be taken into account while adjudging quantum of penalty"))

The factors to be taken into account while adjudging the quantum of penalty are: (a) the amount of disproportionate gain or unfair advantage, wherever quantifiable, made as a result of the default; (b) the amount of loss caused to an investor or group of investors as a result of the default; and (c) the repetitive nature of the default.
